# Lab 2.0: Embeddings com Ollama (Local)

**Objetivo**: Entender como gerar embeddings localmente usando Ollama.

**Comparativo**:
- ❌ **Antes (Lab 1.2)**: Dependência de APIs externas (OpenAI, Google)
- ✅ **Agora (Lab 2.0)**: Modelo local com Ollama, sem custos de API, controle total

**Hardware recomendado**:
- GPU com 4GB+ VRAM (NVIDIA ideal para melhor performance)
- CPU fallback: Funciona perfeitamente, apenas mais lento

**Stack Docker Compose:**
- Ollama na porta 11434
- Modelos disponíveis: nomic-embed-text, mxbai-embed-large, all-minilm

## 1. Setup e Configuração

In [1]:
import os
import json
import time
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Dict, Tuple, Optional

# Carregamento de variáveis de ambiente
from dotenv import load_dotenv

# Para requisições HTTP (Ollama é OpenAI-compatible)
import requests
from requests.exceptions import RequestException, ConnectionError, Timeout

# Logging
from loguru import logger
import logging

# Configurar logger
logger.remove()
logger.add(lambda msg: print(msg, end=''), format="<level>{level: <8}</level> | {message}")

print("✅ Imports principais OK")

✅ Imports principais OK


In [2]:
# Carregar variáveis de ambiente
env_path = Path('/home/jovyan').joinpath('.env').resolve()
if not env_path.exists():
    env_path = Path.cwd().joinpath('.env').resolve()

if env_path.exists():
    load_dotenv(env_path)
    logger.info(f"✅ Carregou .env de: {env_path}")
else:
    logger.warning(f"⚠️  .env não encontrado em: {env_path}")

# Variáveis de configuração
OLLAMA_API_URL = os.getenv('OLLAMA_API_URL', 'http://ollama:11434')
OLLAMA_MODEL = os.getenv('OLLAMA_MODEL', 'nomic-embed-text')
DATA_PATH = Path(os.getenv('DATA_PATH', '/home/jovyan/work/data'))
BATCH_SIZE = int(os.getenv('BATCH_SIZE', 32))

logger.info(f"Ollama URL: {OLLAMA_API_URL}")
logger.info(f"Modelo padrão: {OLLAMA_MODEL}")
logger.info(f"Batch size: {BATCH_SIZE}")

# Criar diretório de dados se não existir
DATA_PATH.mkdir(parents=True, exist_ok=True)

WARNING  | ⚠️  .env não encontrado em: /home/jovyan/.env
INFO     | Ollama URL: http://ollama:11434
INFO     | Modelo padrão: nomic-embed-text
INFO     | Batch size: 32


## 2. Health Check: Verificar Conexão com Ollama

In [4]:
def health_check_ollama(api_url: str, timeout: int = 5) -> bool:
    """
    Verifica se Ollama está rodando e acessível.
    
    Tenta 3 vezes com backoff exponencial antes de desistir.
    """
    try:
        logger.info(f"🔄 Verificando Ollama em {api_url}...")
        
        for tentativa in range(1, 4):
            try:
                response = requests.get(
                    f"{api_url}/api/tags",
                    timeout=timeout
                )
                if response.status_code == 200:
                    models = response.json()
                    model_count = len(models.get('models', []))
                    logger.success(f"✅ Ollama está online! Modelos disponíveis: {model_count}")
                    if model_count > 0:
                        for model in models['models']:
                            logger.info(f"   • {model['name']} ({model['details']['parameter_size']})")
                    return True
            except (ConnectionError, Timeout) as e:
                if tentativa < 3:
                    espera = 2 ** tentativa  # backoff: 2, 4, 8 segundos
                    logger.warning(f"❌ Tentativa {tentativa}/3 falhou. Esperando {espera}s...")
                    time.sleep(espera)
                else:
                    logger.error(f"❌ Ollama não acessível após 3 tentativas")
                    return False
    except Exception as e:
        logger.error(f"❌ Erro ao verificar Ollama: {e}")
        return False

# Executar health check
ollama_online = health_check_ollama(OLLAMA_API_URL)

if not ollama_online:
    logger.error(f"""
    ❌ Ollama não está acessível!
    
    Para iniciar Ollama via Docker Compose:
    1. docker-compose up -d
    2. Verifique a URL: {OLLAMA_API_URL}
    3. Baixe modelos: docker exec ollama-embeddings ollama pull nomic-embed-text
    """)

INFO     | 🔄 Verificando Ollama em http://ollama:11434...
SUCCESS  | ✅ Ollama está online! Modelos disponíveis: 3
INFO     |    • all-minilm:latest (23M)
INFO     |    • mxbai-embed-large:latest (334M)
INFO     |    • nomic-embed-text:latest (137M)


## 3. Função de Embeddings com Ollama

In [5]:
class OllamaEmbedder:
    """
    Wrapper para gerar embeddings via Ollama.
    
    Características:
    - Usa API nativa do Ollama
    - Suporta múltiplos modelos (nomic-embed-text, mxbai-embed-large, all-minilm)
    - Suporta batch processing
    - Retorna embedding + metadata (dimensionalidade, tempo)
    """
    
    def __init__(
        self,
        api_url: str,
        model: str = 'nomic-embed-text',
        timeout: int = 30,
        batch_size: int = 32
    ):
        self.api_url = api_url
        self.model = model
        self.timeout = timeout
        self.batch_size = batch_size
        self.total_tokens = 0
        
    def embed_query(self, text: str, return_metadata: bool = False) -> Tuple[List[float], Optional[Dict]]:
        """
        Gera embedding para um texto único.
        
        Args:
            text: Texto para embeddar
            return_metadata: Retornar info de latência e dimensão
            
        Returns:
            (embedding, metadata) onde metadata é {'dim': int, 'latency_ms': float}
        """
        start_time = time.time()
        
        try:
            payload = {
                'model': self.model,
                'input': text
            }
            
            response = requests.post(
                f"{self.api_url}/api/embed",
                json=payload,
                timeout=self.timeout
            )
            
            if response.status_code != 200:
                logger.error(f"API error: {response.text}")
                return None, None
            
            data = response.json()
            embedding = data['embeddings'][0]
            
            # Metadata
            latency_ms = (time.time() - start_time) * 1000
            metadata = {
                'dim': len(embedding),
                'latency_ms': latency_ms,
                'model': self.model
            }
            
            if return_metadata:
                return embedding, metadata
            return embedding, None
            
        except Exception as e:
            logger.error(f"Erro ao gerar embedding: {e}")
            return None, None
    
    def embed_batch(self, texts: List[str], return_metadata: bool = False) -> Tuple[List[List[float]], Optional[List[Dict]]]:
        """
        Gera embeddings para múltiplos textos (com batch processing).
        
        Args:
            texts: Lista de textos
            return_metadata: Retornar info de latência
            
        Returns:
            ([embeddings], [metadata])
        """
        embeddings = []
        metadata_list = []
        
        total_batches = (len(texts) + self.batch_size - 1) // self.batch_size
        
        for i in range(0, len(texts), self.batch_size):
            batch = texts[i:i + self.batch_size]
            batch_num = (i // self.batch_size) + 1
            
            logger.info(f"🔄 Processando batch {batch_num}/{total_batches}...")
            
            start_time = time.time()
            
            try:
                # Ollama não suporta batch nativamente, processa um por vez
                for text in batch:
                    payload = {
                        'model': self.model,
                        'input': text
                    }
                    
                    response = requests.post(
                        f"{self.api_url}/api/embed",
                        json=payload,
                        timeout=self.timeout
                    )
                    
                    if response.status_code == 200:
                        data = response.json()
                        embeddings.append(data['embeddings'][0])
                    else:
                        logger.error(f"Erro no batch {batch_num}: {response.text}")
                
                latency_ms = (time.time() - start_time) * 1000
                metadata_list.append({
                    'batch_num': batch_num,
                    'texts_in_batch': len(batch),
                    'latency_ms': latency_ms,
                    'avg_latency_per_text': latency_ms / len(batch)
                })
            
            except Exception as e:
                logger.error(f"Erro ao processar batch {batch_num}: {e}")
        
        if return_metadata:
            return embeddings, metadata_list
        return embeddings, None

embedder = OllamaEmbedder(
    api_url=OLLAMA_API_URL,
    model=OLLAMA_MODEL,
    batch_size=BATCH_SIZE
)

logger.success("✅ OllamaEmbedder instanciado")

SUCCESS  | ✅ OllamaEmbedder instanciado


## 4. Teste: Gerar um Embedding Simples

In [6]:
# Teste com textos de exemplo (mesmos do Lab 1.2!)
test_texts = [
    'O gato é um animal doméstico',
    'O gato é um felino de estimação',
    'A programação é importante para engenheiros de software'
]

logger.info("🧪 Gerando embeddings de teste...")

embeddings = []
for idx, text in enumerate(test_texts, 1):
    emb, meta = embedder.embed_query(text, return_metadata=True)
    if emb:
        embeddings.append(emb)
        logger.success(
            f"✅ Texto {idx}: {text[:40]}...\n"
            f"   Dimensão: {meta['dim']} | Latência: {meta['latency_ms']:.2f}ms"
        )
    else:
        logger.error(f"❌ Falha ao processar texto {idx}")

if len(embeddings) == len(test_texts):
    logger.success(f"\n✅ Todos os {len(embeddings)} embeddings gerados com sucesso!")

INFO     | 🧪 Gerando embeddings de teste...
SUCCESS  | ✅ Texto 1: O gato é um animal doméstico...
   Dimensão: 768 | Latência: 2134.34ms
SUCCESS  | ✅ Texto 1: O gato é um animal doméstico...
   Dimensão: 768 | Latência: 2134.34ms
SUCCESS  | ✅ Texto 2: O gato é um felino de estimação...
   Dimensão: 768 | Latência: 4343.85ms
SUCCESS  | ✅ Texto 2: O gato é um felino de estimação...
   Dimensão: 768 | Latência: 4343.85ms
SUCCESS  | ✅ Texto 3: A programação é importante para engenhei...
   Dimensão: 768 | Latência: 1436.15ms
SUCCESS  | 
✅ Todos os 3 embeddings gerados com sucesso!
SUCCESS  | ✅ Texto 3: A programação é importante para engenhei...
   Dimensão: 768 | Latência: 1436.15ms
SUCCESS  | 
✅ Todos os 3 embeddings gerados com sucesso!


## 5. Verificar Similaridade (igual ao Lab 1.2)

In [7]:
def cosine_similarity(v1: List[float], v2: List[float]) -> float:
    """
    Calcula similaridade de cosseno entre dois vetores.
    """
    v1 = np.array(v1)
    v2 = np.array(v2)
    return float(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))

# Calcular similaridades
if len(embeddings) >= 3:
    sim_1_2 = cosine_similarity(embeddings[0], embeddings[1])
    sim_1_3 = cosine_similarity(embeddings[0], embeddings[2])
    
    logger.info(
        f"""
    📊 Similaridades de Cosseno (Ollama Local):
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    Texto 1 vs Texto 2 (esperado: alto): {sim_1_2:.4f}
    Texto 1 vs Texto 3 (esperado: baixo): {sim_1_3:.4f}
    Diferença:                            {(sim_1_2 - sim_1_3):.4f}
    """
    )
    
    if sim_1_2 > sim_1_3:
        logger.success("✅ Modelo conseguiu distinguir conceitos!")
    else:
        logger.warning("⚠️  Similaridades não diferenciam bem os conceitos")

INFO     | 
    📊 Similaridades de Cosseno (Ollama Local):
    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
    Texto 1 vs Texto 2 (esperado: alto): 0.8278
    Texto 1 vs Texto 3 (esperado: baixo): 0.5894
    Diferença:                            0.2384
    
SUCCESS  | ✅ Modelo conseguiu distinguir conceitos!


## 6. Resumo: Lab 2.0

In [8]:
summary = f"""
    ╔════════════════════════════════════════════════════════════╗
    ║           LAB 2.0: EMBEDDINGS COM OLLAMA                    ║
    ╠════════════════════════════════════════════════════════════╣
    ║                                                            ║
    ║ ✅ ENTREGÁVEIS NESTE LAB:                                 ║
    ║                                                            ║
    ║ 1. Conexão com Ollama (Docker Compose)                    ║
    ║ 2. Classe OllamaEmbedder (single + batch processing)      ║
    ║ 3. Geração de embeddings 100% LOCAL (sem APIs remotas)    ║
    ║ 4. Cálculo de latência e dimensionalidade                 ║
    ║ 5. Comparação semântica (cosine similarity)               ║
    ║ 6. Suporte a GPU NVIDIA e CPU                              ║
    ║                                                            ║
    ║ 📊 PRÓXIMOS PASSOS:                                       ║
    ║                                                            ║
    ║ → Lab 2.1: Integração Ollama + Qdrant                     ║
    ║ → Lab 2.2: RAG (Retrieval Augmented Generation)          ║
    ║                                                            ║
    ║ 💡 PONTOS PEDAGÓGICOS:                                    ║
    ║                                                            ║
    ║ • Zero custo de API (após setup inicial)                  ║
    ║ • Múltiplos modelos disponíveis (nomic, mxbai, all-minilm) ║
    ║ • Performance excelente com GPU NVIDIA                    ║
    ║ • Dados não saem de casa (compliance/LGPD)                ║
    ║ • Ollama mais moderno e ativo que LM Studio          ║
    ║                                                            ║
    ╚════════════════════════════════════════════════════════════╝
    """

print(summary)

logger.success("\n✅ Lab 2.0 completo! Avance para Lab 2.1")


    ╔════════════════════════════════════════════════════════════╗
    ║           LAB 2.0: EMBEDDINGS COM OLLAMA                    ║
    ╠════════════════════════════════════════════════════════════╣
    ║                                                            ║
    ║ ✅ ENTREGÁVEIS NESTE LAB:                                 ║
    ║                                                            ║
    ║ 1. Conexão com Ollama (Docker Compose)                    ║
    ║ 2. Classe OllamaEmbedder (single + batch processing)      ║
    ║ 3. Geração de embeddings 100% LOCAL (sem APIs remotas)    ║
    ║ 4. Cálculo de latência e dimensionalidade                 ║
    ║ 5. Comparação semântica (cosine similarity)               ║
    ║ 6. Suporte a GPU NVIDIA e CPU                              ║
    ║                                                            ║
    ║ 📊 PRÓXIMOS PASSOS:                                       ║
    ║                                                            ║
  